In [1]:
from tqdm import tqdm
tqdm.pandas()
from glob import glob

import json
import csv
import numpy as np
import pandas as pd
from collections import Counter

import rdflib
from rdflib import Graph
from data.data import CollectionAccessor, ImageHandler

from search import Search, Randomiser

In [2]:
def init_DMG():
    image_folder = "./data/images/DMG"
    image_handler = ImageHandler(image_folder=image_folder, keep_prefix=False)
    print("image paths loaded...")

    time_stamp, pub_file, priv_file = CollectionAccessor.get_latest_dump("./data/dumps")
    print(CollectionAccessor.get_latest_dump("./data/dumps"))

    
    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_"+time_stamp,
                creation_timestamp=time_stamp)
    df = CollectionAccessor.get_DMG(pub_path=pub_file, #get_latest("./data/dumps", contains="public"),
                                     priv_path=priv_file, #get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    
    rand = Randomiser(df, name="Randomiser")
    rand2 = Randomiser(df, name="Randomiser")
    s = Search([rand, rand2])
    return image_handler, df, s

im, dmg, dmg_search = init_DMG()

image paths loaded...
./data/dumps/API_dump_public_2025-05-13.json ./data/dumps/API_dump_private_2025-05-13.json
./data/dumps/API_dump_public_2025-05-13.json ./data/dumps/API_dump_private_2025-05-13.json
('2025-05-13', './data/dumps/API_dump_public_2025-05-13_extracted.csv', './data/dumps/API_dump_private_2025-05-13_extracted.csv')


100%|████████████████████████████████| 24824/24824 [00:36<00:00, 677.21it/s]


In [20]:
recs = dmg.sample(4)
searcher_ids = [s.id for s in dmg_search.searchers[:1]]

In [24]:
dmg_search(recs, searcher_ids=searcher_ids)

object_number
1992-0004_0-2    0.776923
1992-0004_1-2    0.357817
1992-0004_2-2    0.667643
4521             0.466348
5051             0.770696
                   ...   
2022-0028        0.341355
2022-0025_0-3    0.363114
2022-0025_1-3    0.415226
2022-0025_2-3    0.771660
2022-0025_3-3    0.559759
Length: 24824, dtype: float64

---
# DEV FUNCTIONS

In [ ]:
def search_collection(collection_id, object_ids, concept, model_list):
    cur_coll = get_collection(collection_id)
    object_ids = parse_id_list(object_ids)
    model_list = parse_id_list(model_list)
    cur_search = searches[collection_id]
    print(object_ids)
    scores = cur_search(object_ids)


object_ids = "1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"
object_ids = parse_id_list(object_ids)
object_ids
recs = dmg.loc[object_ids]

In [ ]:
# searcher_scores = [cur_s(recs) for cur_s in dmg_search.searchers]
# searcher_scores = pd.DataFrame({cur_s.name: cur_s for cur_s in searcher_scores})
# searcher_scores.loc[recs.index] = 0.


# searcher_scores.round(3)

In [ ]:
dmg.coll.filter("doos")

---
# TESTING ROUTES

In [ ]:
!curl "http://0.0.0.0:8080/collections"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/object-details?object_ids=1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/models"

In [ ]:
!curl "http://0.0.0.0:8080/moon?ISO_8601_datetime=2024-05-13&lat_degrees=51.05&long_degrees=3.71"

In [ ]:
# default params for search
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b""

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/order?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&skip=200&limit=100&reverse=true"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/sample?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=12"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-13/search/order/filter?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&filter_text="doos"&skip=200&limit=100"

---

In [ ]:
import requests as rq
base = "http://0.0.0.0:8080/"
path = "moon"
for d in range(10, 30):
    q = dict(ISO_8601_datetime=f"2024-05-{d}", lat_long_degrees="51.05,3.71")
    print(q)
    resp = rq.get(base+path, params=q)
    
    print(resp.json())